## Prerequisites

1. ✅ foundation/00-setup-postgres-schema.ipynb
2. ✅ foundation/02-rag-postgresql-persistent.ipynb (generates initial embeddings)

## Configuration

In [ ]:
# Synthetic generation approach
GENERATION_APPROACH = "blended"  # 'llm' or 'template' or 'blended'
LLM_MODEL = "gpt-3.5-turbo"      # For LLM generation
QUESTIONS_PER_CHUNK = 3          # How many questions per document chunk

# Curation
BATCH_SIZE = 10  # How many to show at once
INTERACTIVE_MODE = True  # Set to False for automated labeling

## Phase 1: Synthetic Generation

### Option A: LLM-Based Generation

In [ ]:
# TODO: LLM-based question generation
# For each chunk from dataset:
#   1. Call LLM: "Generate 3 questions that would be answered by this chunk:"
#   2. Parse response, extract questions
#   3. Append (question, chunk_id, source='llm') to candidate list

pass

### Option B: Template-Based Generation

In [ ]:
# TODO: Template-based question generation
# For each chunk:
#   1. Extract named entities (persons, places, organizations)
#   2. Extract facts (who did what, when, where)
#   3. Generate questions using templates:
#      - "Who is {entity}?"
#      - "What did {entity} do?"
#      - "When did {event} happen?"
#   4. Append (question, chunk_id, source='template') to candidate list

pass

### Deduplicate Candidates

In [ ]:
# TODO: Deduplicate
# 1. For each pair of questions, compute similarity
# 2. If similarity > 0.8, keep only one
# 3. Randomize order for curation

pass

## Phase 2: Interactive Human Curation

Curator reviews each generated question and provides feedback.

In [ ]:
# TODO: Interactive curation loop
# For each question in candidate list:
#   1. Show: question, source chunks (full text), suggested chunk IDs
#   2. Prompt curator:
#      - [G]ood - accept as-is
#      - [B]ad - reject
#      - [A]mbiguous - needs clarification
#      - [R]eject - don't want this question
#      - [N]otes - add notes
#   3. If [G]ood:
#      - Insert to evaluation_groundtruth with quality_rating='good'
#   4. If [A]mbiguous:
#      - Let curator edit question and relevant_chunk_ids
#      - Insert with quality_rating='ambiguous'
#   5. If [B]ad or [R]eject:
#      - Insert with quality_rating='rejected'
#   6. If [N]otes:
#      - Capture human_notes field

pass

## Store Results

In [ ]:
# TODO: Store curated ground-truth
# INSERT curated questions into evaluation_groundtruth table:
# - question (string)
# - source_type ('llm_generated' or 'template_based' or 'human_written')
# - relevant_chunk_ids (integer array)
# - quality_rating ('good', 'ambiguous', 'rejected')
# - human_notes (optional curator comments)

pass

## Summary

In [ ]:
# TODO: Show summary statistics
# - Total questions curated
# - Distribution by quality_rating
# - Distribution by source_type
# - Average chunk_ids per question
# - Time spent on curation

pass